In [131]:
#wordnet in nltk
from nltk.corpus import wordnet as wn
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
#import classification report
from sklearn.metrics import classification_report, confusion_matrix
nltk.download('wordnet')
import numpy as np
import gensim
from nltk.data import find

from torch import nn
import torch
from torch.nn import functional as F

#random forest
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package wordnet to /home/dorin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
#read imdb/imdb.neg
with open('imdb/imdb.neg', 'r') as f:
    neg = f.readlines()
#read imdb/imdb.pos
with open('imdb/imdb.pos', 'r') as f:
    pos = f.readlines()

#load nltk word2vec_sample

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [62]:
# given model and two words, find cosine similarity

def cosine_similarity(model, word1, word2):
    vec1 = model[word1]
    vec2 = model[word2]
    similarity = vec1 @ vec2 / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    #similarity = model.similarity(word1, word2)
    return similarity

In [114]:
# use ngrams to develop a word predictor
# given a word, predict the next word

def ngram_model(corpus, ngram_size):
    # create all the ngrams present in the corpus of size ngram_size
    ngrams = {}
    for line in corpus:
        tokens = line.split()
        for i in range(len(tokens) - ngram_size):
            ngram = tuple(tokens[i:i+ngram_size])
            if ngram not in ngrams:
                ngrams[ngram] = 1
            else:
                ngrams[ngram] += 1
    return ngrams


In [117]:
ngrams = ngram_model(neg+pos, 2)

In [119]:
# given a word, predict the next word
def predict_next_word(word, ngrams):
    # find all ngrams that start with the given word
    candidates = []
    for ngram in ngrams:
        if ngram[0] == word:
            candidates.append(ngram)
    # find the most common ngram
    max_count = 0
    max_ngram = None
    for candidate in candidates:
        if ngrams[candidate] > max_count:
            max_count = ngrams[candidate]
            max_ngram = candidate
    return max_ngram[1]

In [ ]:
def predict_next_word(words, ngrams):
    # find all ngrams that start with the given words
    candidates = []
    for ngram in ngrams:
        for i in range(len(words)):
            if ngram[i] != words[i]:
                break

In [122]:
def predict_next_n_words(word, ngrams, n):
    sentence = [word]
    for i in range(n):
        word = predict_next_word(word, ngrams)
        sentence.append(word)
    return " ".join(sentence)

In [124]:
predict_next_n_words("good", ngrams, 10)

'good as a great movie , but not a great movie'

In [105]:
def predict_next_word(vectorizer, word):
    # get the index of the word in the vocabulary
    word_idx = vectorizer.vocabulary_.get(word, -1)
    if word_idx == -1:
        return None
    # count the amount of times word is present in the vectorizer as the first element of the ngram
    word_count = vectorizer.transform([word]).toarray()[0][word_idx]
    # get the ngrams that start with the word
    ngrams = vectorizer.get_feature_names()[word_idx:word_idx + 100]
    # get the counts of the ngrams
    ngram_counts = vectorizer.transform([word]).toarray()[0][word_idx:word_idx + 100]
    # get the next word
    next_word = ngrams[np.argmax(ngram_counts)]
    return next_word

predict_next_word(vectorizer, 'forget')

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [113]:
# get the amount of times "good" appears in the vectorizer


0


In [61]:
cosine_similarity(model, "civilized", "French")

0.14630209

In [127]:
# RNN for sentiment analysis classification
train_data = neg[:1000] + pos[:1000]
test_data = neg[1000:2000] + pos[1000:2000]
train_labels = [0] * 1000 + [1] * 1000
test_labels = [0] * 1000 + [1] * 1000

In [179]:
# embed the sentences using word2vec
def embed_sentences(sentences, model):
    embedded_sentences = []
    for sentence in sentences:
        embedded_sentence = []
        for word in sentence.split():
            if word in model:
                embedded_sentence.append(model[word])
        embedded_sentences.append(embedded_sentence)
    return embedded_sentences

embedded_train_data = embed_sentences(train_data, model)
embedded_test_data = embed_sentences(test_data, model)

longest_sentence = max(embedded_train_data, key=len)

# convert sentences to tensor
def pad_and_convert_sentences_to_tensor_padded(embedded_sentences):
    # pad the sentences
    for sentence in embedded_sentences:
        while len(sentence) < len(longest_sentence):
            sentence.append(np.zeros(300))
    # convert the sentences to tensors
    tensor_sentences = []
    for sentence in embedded_sentences:
        tensor_sentences.append(torch.tensor(sentence))
    tensor_sentences = torch.stack(tensor_sentences).float()
    return tensor_sentences

tensor_train_data = pad_and_convert_sentences_to_tensor_padded(embedded_train_data)
tensor_test_data = pad_and_convert_sentences_to_tensor_padded(embedded_test_data)


In [279]:
# create RNN class for the classification task
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        # initialize the hidden state
        self.hidden_size = hidden_size
        self.hidden = torch.zeros(1, 1, hidden_size)
        # initialize the layers
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # run the RNN layer
        out, self.hidden = self.rnn(x.view(1, 1, -1), self.hidden)
        # run the fully connected layer
        out = self.fc(out.view(1, -1))
        return out

In [281]:
rnn = RNN(300, 100, 1)
rnn(tensor_train_data)

RuntimeError: Expected hidden size (1, 31, 100), got [1, 1, 100]